In [8]:
### setup
library(RCurl) # = getForm()
library(jsonlite)

### functions
LOG = function(txt){
    write(txt, file="log.txt")
}

### read assets
cData = fromJSON('data/championsData.json')
cData.keys = c()
for(champ in cData$data){
    cData.keys[champ$key] = champ$name
}
cData.get = function(key){
    return(cData$data[which(names(cData.keys) == key)])
}
iData = fromJSON('data/itemsData.json')

### LoL-API
key = "?api_key=RGAPI-f8512751-f6c2-4dcc-9858-43a2ff3a809e"
api_url = "https://kr.api.riotgames.com"

In [9]:
cData.get(875)

$Sett
$Sett$version
[1] "10.4.1"

$Sett$id
[1] "Sett"

$Sett$key
[1] "875"

$Sett$name
[1] "세트"

$Sett$title
[1] "우두머리"

$Sett$blurb
[1] "점점 세력을 더해 가는 아이오니아 범죄 세계의 우두머리 세트는 녹서스와의 전쟁이 끝남과 동시에 두각을 드러내기 시작했다. 나보리 투기장의 신인 투사로 등장한 그는 무지막지한 힘과 상상을 초월하는 맷집으로 금세 유명인사가 되었고, 동네 싸움꾼에서 한때 그가 싸웠던 투기장을 지배하는 정상의 자리까지 올라섰다."

$Sett$info
$Sett$info$attack
[1] 8

$Sett$info$defense
[1] 5

$Sett$info$magic
[1] 1

$Sett$info$difficulty
[1] 2


$Sett$image
$Sett$image$full
[1] "Sett.png"

$Sett$image$sprite
[1] "champion3.png"

$Sett$image$group
[1] "champion"

$Sett$image$x
[1] 48

$Sett$image$y
[1] 48

$Sett$image$w
[1] 48

$Sett$image$h
[1] 48


$Sett$tags
[1] "Fighter" "Tank"   

$Sett$partype
[1] "투지"

$Sett$stats
$Sett$stats$hp
[1] 600

$Sett$stats$hpperlevel
[1] 93

$Sett$stats$mp
[1] 0

$Sett$stats$mpperlevel
[1] 0

$Sett$stats$movespeed
[1] 340

$Sett$stats$armor
[1] 37

$Sett$stats$armorperlevel
[1] 4

$Sett$stats$spellblock
[1] 32

$Sett$stats$spellblockperlevel
[1] 1.25

$Sett$stats$attackrange
[1] 125

$Sett$stats$hpregen
[1] 8

$Sett$stats$hpregenperlevel
[1] 0.75

$Sett$stats$mpregen
[1] 0

$Sett$stats$mpregenperlevel
[1] 0

$Sett$stats$crit
[1] 0

$Sett$stats$critperlevel
[1] 0

$Sett$stats$attackdamage
[1] 60

$Sett$stats$attackdamageperlevel
[1] 4

$Sett$stats$attackspeedperlevel
[1] 1.75

$Sett$stats$attackspeed
[1] 0.625

In [52]:
name = "김옥티/"

In [53]:
### get {NAME} summoner
operation = '/lol/summoner/v4/summoners/by-name/'
url = paste0(api_url, operation, name, key)
LOG(url)
result = fromJSON(getForm(url, query=""))
write(toJSON(result), file=paste0("data/", name,"summoner.json"))

In [3]:
tier = 'BRONZE/'

In [14]:
### get {TIER} Summoner List
operation = '/lol/league-exp/v4/entries/'
queue = 'RANKED_SOLO_5x5/'
division = 'I'
page = '&page=1'
url = paste0(api_url, operation, queue, tier, division, key, page)
result = fromJSON(getForm(url, query=""))
LOG(url)
write(toJSON(result), file=paste0("data/", tier,"playerList.json"))

In [4]:
target = tier
result = fromJSON(paste0("data/",tier,"playerList.json"))
# result = fromJSON(paste0("data/",name,"summoner.json"))

In [10]:
detach("package:rjson")
library(jsonlite)

ERROR: Error in detach("package:rjson"): invalid 'name' argument


In [16]:
### NAME
update = function(){
    ### mkdir
    dir.create(paste0("data/", target))

    ### get Summoner
    data.summoner = result

    ### get Summoner's League - Tier
    operation = '/lol/league/v4/entries/by-summoner/'
    url = paste0(api_url, operation, data.summoner$id, key)
    LOG(url)
    write(getForm(url, query=""), file=paste0("data/",target,"/summoner_league.json"))
    Sys.sleep(0.05)

    ### get Champion Mastery
    operation = "/lol/champion-mastery/v4/champion-masteries/by-summoner/"
    url = paste0(api_url, operation, data.summoner$id, key)
    LOG(url)
    data.mastery = fromJSON(getForm(url, query = ""))
    Sys.sleep(0.05)
    write(toJSON(data.mastery), file=paste0("data/",target,"/mastery.json"))

    ### get MatchList
    operation = "/lol/match/v4/matchlists/by-account/"
    # champion
    queue = "&queue=420&queue=430&queue=440" # 랭크게임, 일반게임, 자유랭
    season = "&season=13"
    # endTime
    # beginTime
    matchLength = 50
    endIndex = paste0("&endIndex=",matchLength)
    beginIndex = "&beginIndex=0"
    url = paste0(api_url, operation, data.summoner$accountId, key, queue, season, endIndex, beginIndex)
    data.matchList = fromJSON(getForm(url, query=""))$matches
    write(toJSON(data.matchList), file=paste0("data/",target,"/matchList.json"))

    Sys.sleep(20/12)

    ### get match
    data.match = c()
    data.match.participants = c()
    data.match.participants.info = c()
    data.match.participants.stats = c()

    print("start Loop")
    for(j in 1:matchLength){
        index = j
        currentMatch = data.matchList[index, ]
        operation = '/lol/match/v4/matches/'
        id = currentMatch$gameId
        url = paste0(api_url, operation, id, key)

        # use rjson rather than jsonlite
        detach("package:jsonlite")
        library(rjson)

        data.match.raw = getForm(url, query="")
        data.match.participants[[index]] = fromJSON(data.match.raw)[12:13]

        # use jsonlite rather than rjson
        detach("package:rjson")
        library(jsonlite)

        data.match[[index]] = c(fromJSON(data.match.raw)[1:11])
        p = data.match.participants[[index]][[1]]
        data.match.participants.stats[[index]] = lapply(p, function(x) data.frame(x$stats))
        p = data.match.participants[[index]][[2]]
        data.match.participants.info[[index]] = lapply(p, function(x) data.frame(x$player))

        print(paste0('- finished: ', index))
        Sys.sleep(10/12)
    }
    data.match.participants = lapply(data.match.participants, function(x) x$participants)
    write(toJSON(data.match.participants), file=paste0("data/",target,"/match_participants.json"))
    write(toJSON(data.match.participants.stats), file=paste0("data/",target,"/match_participants_stats.json"))
    write(toJSON(data.match.participants.info), file=paste0("data/",target,"/match_participants_info.json"))
    write(toJSON(data.match), file=paste0("data/",target,"/match.json"))
} 

In [9]:
### TIER
start = 154
end = 200
now = start
update = function(start,end){
    for(i in start:end){
        assign("now", i, envir = .GlobalEnv)
        ### mkdir
        dir.create(paste0("data/", target, i))

        ### get Summoner
        operation = "/lol/summoner/v4/summoners/"
        id = result$summonerId[i]
        url = paste0(api_url, operation, id, key)
        page = getForm(url, query="")
        write(page, file=paste0("data/",target, i,"/summoner.json"))
        data.summoner = fromJSON(page)

#         ### get Summoner's League - Tier
#         operation = '/lol/league/v4/entries/by-summoner/'
#         url = paste0(api_url, operation, data.summoner$id, key)
#         LOG(url)
#         write(getForm(url, query=""), file=paste0("data/",target, i,"/summoner_league.json"))
#     	Sys.sleep(0.05)

#         ### get Champion Mastery
#         operation = "/lol/champion-mastery/v4/champion-masteries/by-summoner/"
#         url = paste0(api_url, operation, data.summoner$id, key)
#         LOG(url)
#         data.mastery = fromJSON(getForm(url, query = ""))
#         Sys.sleep(0.05)
#         write(toJSON(data.mastery), file=paste0("data/",target, i,"/mastery.json"))

        ### get MatchList
        operation = "/lol/match/v4/matchlists/by-account/"
        # champion
        queue = "&queue=420&queue=430&queue=440" # 랭크게임, 일반게임, 자유랭
        season = "&season=13"
        # endTime
        # beginTime
        matchLength = 20
        endIndex = paste0("&endIndex=",matchLength)
        beginIndex = "&beginIndex=0"
        url = paste0(api_url, operation, data.summoner$accountId, key, queue, season, endIndex, beginIndex)
        data.matchList = fromJSON(getForm(url, query=""))$matches
        write(toJSON(data.matchList), file=paste0("data/",target, i,"/matchList.json"))

        Sys.sleep(20/12)

        ### get match
        data.match = c()
        data.match.participants = c()
        data.match.participants.info = c()
        data.match.participants.stats = c()

        print("start Loop")
        for(j in 1:matchLength){
            index = j
            currentMatch = data.matchList[index, ]
            operation = '/lol/match/v4/matches/'
            id = currentMatch$gameId
            url = paste0(api_url, operation, id, key)

            # use rjson rather than jsonlite
            detach("package:jsonlite")
            library(rjson)

            data.match.raw = getForm(url, query="")
            data.match.participants[[index]] = fromJSON(data.match.raw)[12:13]

            # use jsonlite rather than rjson
            detach("package:rjson")
            library(jsonlite)

            data.match[[index]] = c(fromJSON(data.match.raw)[1:11])
            p = data.match.participants[[index]][[1]]
            data.match.participants.stats[[index]] = lapply(p, function(x) data.frame(x$stats))
            p = data.match.participants[[index]][[2]]
            data.match.participants.info[[index]] = lapply(p, function(x) data.frame(x$player))

            print(paste0('- finished: ', index))
            Sys.sleep(10/12)
        }
        data.match.participants = lapply(data.match.participants, function(x) x$participants)
        write(toJSON(data.match.participants), file=paste0("data/",target, i,"/match_participants.json"))
        write(toJSON(data.match.participants.stats), file=paste0("data/",target, i,"/match_participants_stats.json"))
        write(toJSON(data.match.participants.info), file=paste0("data/",target, i,"/match_participants_info.json"))
        write(toJSON(data.match), file=paste0("data/",target, i,"/match.json"))

        print(paste0('> Finished: ', i))
    }
}

In [60]:
### NAME
update()

ERROR: Error in update(): argument "start" is missing, with no default


In [ ]:
print("이게되네")

In [11]:
### TIER
loop = function(){
    tryCatch({
        update(now, end)
    }, error = function(e) {
        print(e)
        unlink(paste0('data/', target, now), recursive=T);
        Sys.sleep(10); 
        try({
            detach("package:rjson");
            library(jsonlite);
        })
        loop()
    }, silent=T)
}
system.time(loop())[3] # miliSeconds (3600ms = 1h)

Warning message in dir.create(paste0("data/", target, i)):
“'data/BRONZE/154' already exists”


[1] "start Loop"
[1] "- finished: 1"
[1] "- finished: 2"
[1] "- finished: 3"
[1] "- finished: 4"
[1] "- finished: 5"
[1] "- finished: 6"
[1] "- finished: 7"
[1] "- finished: 8"
[1] "- finished: 9"
[1] "- finished: 10"
[1] "- finished: 11"
[1] "- finished: 12"
[1] "- finished: 13"
[1] "- finished: 14"
[1] "- finished: 15"
[1] "- finished: 16"
[1] "- finished: 17"
[1] "- finished: 18"
[1] "- finished: 19"
[1] "- finished: 20"
[1] "> Finished: 154"


Warning message in dir.create(paste0("data/", target, i)):
“'data/BRONZE/155' already exists”


[1] "start Loop"
[1] "- finished: 1"
[1] "- finished: 2"
[1] "- finished: 3"
[1] "- finished: 4"
[1] "- finished: 5"
[1] "- finished: 6"
[1] "- finished: 7"
[1] "- finished: 8"
[1] "- finished: 9"
[1] "- finished: 10"
[1] "- finished: 11"
[1] "- finished: 12"
[1] "- finished: 13"
[1] "- finished: 14"
[1] "- finished: 15"
[1] "- finished: 16"
[1] "- finished: 17"
[1] "- finished: 18"
[1] "- finished: 19"
[1] "- finished: 20"
[1] "> Finished: 155"


Warning message in dir.create(paste0("data/", target, i)):
“'data/BRONZE/156' already exists”


...

Warning message in dir.create(paste0("data/", target, i)):
“'data/BRONZE/199' already exists”


[1] "start Loop"
[1] "- finished: 1"
[1] "- finished: 2"
[1] "- finished: 3"
[1] "- finished: 4"
[1] "- finished: 5"
[1] "- finished: 6"
[1] "- finished: 7"
[1] "- finished: 8"
[1] "- finished: 9"
[1] "- finished: 10"
[1] "- finished: 11"
[1] "- finished: 12"
[1] "- finished: 13"
[1] "- finished: 14"
[1] "- finished: 15"
[1] "- finished: 16"
[1] "- finished: 17"
[1] "- finished: 18"
[1] "- finished: 19"
[1] "- finished: 20"
[1] "> Finished: 199"


Warning message in dir.create(paste0("data/", target, i)):
“'data/BRONZE/200' already exists”


[1] "start Loop"
[1] "- finished: 1"
[1] "- finished: 2"
[1] "- finished: 3"
[1] "- finished: 4"
[1] "- finished: 5"
[1] "- finished: 6"
[1] "- finished: 7"
[1] "- finished: 8"
[1] "- finished: 9"
[1] "- finished: 10"
[1] "- finished: 11"
[1] "- finished: 12"
[1] "- finished: 13"
[1] "- finished: 14"
[1] "- finished: 15"
[1] "- finished: 16"
[1] "- finished: 17"
[1] "- finished: 18"
[1] "- finished: 19"
[1] "- finished: 20"
[1] "> Finished: 200"


elapsed 
11931.91

In [7]:
12772 / 3600

[1] 3.547778

In [ ]:
ppp = system.time(print(12))
ppp[3] 